In [2]:
# 🧩 Imports
import pandas as pd
import requests
from tqdm import tqdm
import time
from datetime import datetime

# 🛠️ Config
ETHERSCAN_API_KEY = "API KEY"  # Replace with your actual key
COMPOUND_CTOKEN_ADDRESSES = {
    "cDAI": "0x5d3a536e4d6dbd6114cc1ead35777bab948e3643",
    "cUSDC": "0x39aa39c021dfbae8fac545936693ac917d5e7563",
    "cETH": "0x4ddc2d193948926d02f9b1fe9e1daa0718270ed5",
    # Add more if needed
}

# 📂 Load wallet list
wallet_df = pd.read_csv("Wallet id.csv")
wallet_list = wallet_df["wallet_id"].tolist()

# 🧪 Helper: Fetch transactions
def get_erc20_tx(wallet_address):
    url = (
        f"https://api.etherscan.io/api"
        f"?module=account"
        f"&action=tokentx"
        f"&address={wallet_address}"
        f"&startblock=0"
        f"&endblock=99999999"
        f"&sort=asc"
        f"&apikey={ETHERSCAN_API_KEY}"
    )
    try:
        response = requests.get(url)
        data = response.json()
        if data["status"] == "1":
            return data["result"]
        else:
            return []
    except:
        return []

# 🧠 Feature Extraction
def extract_features(wallet, txs):
    features = {
        "wallet_id": wallet,
        "total_supply": 0,
        "total_borrow": 0,
        "total_repay": 0,
        "liquidation_count": 0,
        "tx_count": 0,
        "active_days": set()
    }
    for tx in txs:
        to_addr = tx["to"].lower()
        from_addr = tx["from"].lower()
        token_symbol = tx.get("tokenSymbol", "")
        time_stamp = datetime.fromtimestamp(int(tx["timeStamp"])).date()

        features["tx_count"] += 1
        features["active_days"].add(time_stamp)

        # Check for supply (transfer to cToken address)
        if to_addr in COMPOUND_CTOKEN_ADDRESSES.values():
            features["total_supply"] += float(tx["value"]) / (10 ** int(tx["tokenDecimal"]))

        # Check for repay (transfer to cToken address)
        if to_addr in COMPOUND_CTOKEN_ADDRESSES.values() and token_symbol.startswith("c"):
            features["total_repay"] += float(tx["value"]) / (10 ** int(tx["tokenDecimal"]))

        # Borrow and liquidation must be parsed via input data or deeper parsing (skipped in basic version)
        # For now we approximate by txs to/from cTokens only

    features["active_days"] = len(features["active_days"])
    features["net_balance"] = features["total_supply"] - features["total_borrow"]
    return features

# 🌀 Main loop
all_features = []

for wallet in tqdm(wallet_list):
    txs = get_erc20_tx(wallet)
    wallet_features = extract_features(wallet, txs)
    all_features.append(wallet_features)
    time.sleep(0.25)  # avoid rate limit

features_df = pd.DataFrame(all_features)

# 🧼 Fill missing/empty
features_df.fillna(0, inplace=True)

# 🔄 Normalize Features
def normalize_column(col):
    if features_df[col].max() == features_df[col].min():
        return [0.5] * len(features_df)
    return (features_df[col] - features_df[col].min()) / (features_df[col].max() - features_df[col].min())

features_df["norm_supply"] = normalize_column("total_supply")
features_df["norm_borrow"] = normalize_column("total_borrow")
features_df["norm_repay"] = normalize_column("total_repay")
features_df["norm_liquidation"] = normalize_column("liquidation_count")
features_df["norm_net_balance"] = normalize_column("net_balance")
features_df["norm_tx_count"] = normalize_column("tx_count")
features_df["norm_active_days"] = normalize_column("active_days")

# 🧮 Scoring (0–1000)
features_df["score"] = (
    0.25 * (1 - features_df["norm_supply"]) +
    0.25 * features_df["norm_borrow"] +
    0.2 * features_df["norm_liquidation"] +
    0.15 * (1 - features_df["norm_net_balance"]) +
    0.15 * (1 - features_df["norm_tx_count"])
) * 1000

features_df["score"] = features_df["score"].astype(int)

# 📤 Save Output
output_df = features_df[["wallet_id", "score"]]
output_df.to_csv("wallet_scores.csv", index=False)

# 📊 Optional: Display top risky wallets
output_df.sort_values("score", ascending=False).head(10)


100%|████████████████████████████████████████████████████████████████████████████████| 103/103 [02:31<00:00,  1.48s/it]


,wallet_id,score
14,0x1e43dacdcf863676a6bec8f7d6896d6252fac669,775
29,0x4c4d05fe859279c91b074429b5fc451182cec745,775
69,0x96bb4447a02b95f1d1e85374cffd565eb22ed2f8,775
6,0x111c7208a7e2af345d36b6d4aace8740d61a3078,774
5,0x104ae61d8d487ad689969a17807ddc338b445416,774
2,0x0795732aacc448030ef374374eaae57d2965c16c,774
1,0x06b51c6882b27cb05e712185531c1f74996dd988,774
37,0x56cc2bffcb3f86a30c492f9d1a671a1f744d1d2f,774
36,0x56ba823641bfc317afc8459bf27feed6eb9ff59f,774
10,0x1724e16cb8d0e2aa4d08035bc6b5c56b680a3b22,774
